In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
from tqdm import tqdm

import numpy as np
import cv2
from torchvision import models
import torch.nn.functional as F

In [8]:
# 1. Уменьшение больших .txt файлов
# ============================================================
def reduce_txt(input_txt, output_txt, max_rows):
    """
    Уменьшает количество строк в .txt файле до max_rows
    с сохранением пропорций классов.
    """
    df = pd.read_csv(
        input_txt,
        sep=" ", header=None,
        names=["filename", "class", "xmin", "ymin", "xmax", "ymax"]
    )

    if len(df) <= max_rows:
        df.to_csv(output_txt, sep=" ", header=False, index=False)
        print(f"{input_txt}: строк {len(df)}, сохранили без изменений.")
        return

    df_small = df.groupby("class", group_keys=False).apply(
        lambda x: x.sample(frac=min(1.0, max_rows/len(df)), random_state=42)
    )
    df_small = df_small.sample(frac=1, random_state=42).reset_index(drop=True)

    df_small.to_csv(output_txt, sep=" ", header=False, index=False)
    print(f"{input_txt}: уменьшено {len(df)} → {len(df_small)}, сохранено {output_txt}")

In [9]:
# 2. Трансформации
# ============================================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [10]:
# 3. Класс датасета
# ============================================================
class CovidCTDataset(Dataset):
    def __init__(self, txt_file, img_dir, transform=None, use_crop=False):
        self.df = pd.read_csv(
            txt_file, sep=" ", header=None,
            names=["filename", "class", "xmin", "ymin", "xmax", "ymax"]
        )
        self.img_dir = img_dir
        self.transform = transform
        self.use_crop = use_crop

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["filename"])
        image = Image.open(img_path).convert("RGB")
        label = int(row["class"])

        if self.use_crop:
            xmin, ymin, xmax, ymax = row[["xmin", "ymin", "xmax", "ymax"]]
            image = image.crop((xmin, ymin, xmax, ymax))

        if self.transform:
            image = self.transform(image)

        return image, label

In [11]:
# 4. DataLoader
# ============================================================
def get_dataloaders(train_file, val_file, test_file, img_dir, batch_size=32):
    train_ds = CovidCTDataset(train_file, img_dir, transform=train_transform)
    val_ds   = CovidCTDataset(val_file, img_dir, transform=val_test_transform)
    test_ds  = CovidCTDataset(test_file, img_dir, transform=val_test_transform)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=0)

    return train_loader, val_loader, test_loader

In [12]:
# 5. Модель DenseNet121
# ============================================================
def get_model(num_classes=3):
    model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
    in_f = model.classifier.in_features
    model.classifier = nn.Linear(in_f, num_classes)
    return model

In [13]:
# 6. Циклы обучения и теста
# ============================================================
def train_one_epoch(model, loader, criterion, optimizer, device, epoch, total_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    pbar = tqdm(loader, desc=f"Epoch [{epoch}/{total_epochs}] Training", unit="batch")
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        pbar.set_postfix(loss=running_loss/total, acc=100*correct/total)

    return float(running_loss/total), float(correct/total)


In [14]:
def evaluate(model, loader, criterion, device, phase="Val"):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc=f"{phase} evaluating", unit="batch"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return float(running_loss/total), float(correct/total)

In [2]:
# Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [ ]:
history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": [], "test_loss": [], "test_acc": []}

In [ ]:
# 7. Основной блок
# ============================================================
# Уменьшаем файлы (если нужно)
reduce_txt("dataset2/train_COVIDx_CT-3A.txt", "train_small.txt", 50000)
reduce_txt("dataset2/val_COVIDx_CT-3A.txt", "val_small.txt", 5000)
reduce_txt("dataset2/test_COVIDx_CT-3A.txt", "test_small.txt", 5000)

img_dir = "dataset2/3A_images"  # путь к папке с картинками
train_file, val_file, test_file = "train_small.txt", "val_small.txt", "test_small.txt"

# DataLoader
train_loader, val_loader, test_loader = get_dataloaders(train_file, val_file, test_file, img_dir, batch_size=16)

# Модель
model = get_model(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Обучение
num_epochs = 20

for epoch in range(1, num_epochs+1):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch, num_epochs)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device, phase="Val")
    test_loss, test_acc = evaluate(model, test_loader, criterion, device, phase="Test")

    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)
    history["test_loss"].append(test_loss)
    history["test_acc"].append(test_acc)

    print(f"Epoch {epoch}/{num_epochs} -> "
            f"Train acc: {train_acc:.2f}, Val acc: {val_acc:.2f}, Test acc: {test_acc:.2f}")
    torch.cuda.empty_cache()
    
best_model_path = "best_densenet_ct.pth"
torch.save(model.state_dict(), best_model_path)
print(f"Модель сохранена в {best_model_path}")

C:\Users\denvd\AppData\Local\Temp\ipykernel_8648\406885135.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_small = df.groupby("class", group_keys=False).apply(
C:\Users\denvd\AppData\Local\Temp\ipykernel_8648\406885135.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_small = df.groupby("class", group_keys=False).apply(


dataset2/train_COVIDx_CT-3A.txt: уменьшено 357518 → 50000, сохранено train_small.txt
dataset2/val_COVIDx_CT-3A.txt: уменьшено 33725 → 5000, сохранено val_small.txt
dataset2/test_COVIDx_CT-3A.txt: уменьшено 33781 → 5000, сохранено test_small.txt


C:\Users\denvd\AppData\Local\Temp\ipykernel_8648\406885135.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_small = df.groupby("class", group_keys=False).apply(
Test evaluating: 100%|██████████| 313/313 [00:55<00:00,  5.64batch/s]


Epoch 1/20 -> Train acc: 0.97, Val acc: 0.92, Test acc: 0.93


Test evaluating: 100%|██████████| 313/313 [00:34<00:00,  9.04batch/s]


...

Test evaluating: 100%|██████████| 313/313 [00:34<00:00,  9.12batch/s]


Epoch 19/20 -> Train acc: 1.00, Val acc: 0.90, Test acc: 0.89


Test evaluating: 100%|██████████| 313/313 [00:54<00:00,  5.74batch/s]

Epoch 20/20 -> Train acc: 1.00, Val acc: 0.81, Test acc: 0.82
Модель сохранена в best_densenet_ct.pth


: 

In [ ]:
# Графики
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history["train_loss"], label="Train Loss")
plt.plot(history["val_loss"], label="Val Loss")
plt.plot(history["test_loss"], label="Test Loss")
plt.title("Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history["train_acc"], label="Train Acc")
plt.plot(history["val_acc"], label="Val Acc")
plt.plot(history["test_acc"], label="Test Acc")
plt.title("Accuracy")
plt.legend()

plt.show()

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [4]:

# Классы (под свой датасет)
class_names = ["Normal", "CAP", "COVID-19"]

def get_cam_bbox(cam, threshold=0.5):
    """
    Возвращает координаты прямоугольника по Grad-CAM карте
    threshold: порог выделения активной зоны (0-1)
    """
    mask = cam > threshold
    if not np.any(mask):
        return None  # если модель ничего не выделила

    ys, xs = np.where(mask)
    xmin, xmax = xs.min(), xs.max()
    ymin, ymax = ys.min(), ys.max()
    return (xmin, ymin, xmax, ymax)

def predict_image_with_bbox(model, image_path, device, target_layer="features.denseblock4"):
    img = Image.open(image_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)

    # Сохраняем активации и градиенты
    activations, gradients = {}, {}

    def forward_hook(module, inp, out):
        activations["value"] = out

    def backward_hook(module, grad_in, grad_out):
        gradients["value"] = grad_out[0]

    # Вешаем хуки
    layer = dict([*model.named_modules()])[target_layer]
    fwd_handle = layer.register_forward_hook(forward_hook)
    bwd_handle = layer.register_backward_hook(backward_hook)

    model.eval()
    outputs = model(input_tensor)  # без torch.no_grad()!
    probs = torch.softmax(outputs, dim=1)
    conf, pred = torch.max(probs, 1)

    # Обратный проход для CAM
    model.zero_grad()
    outputs[0, pred.item()].backward()

    grads = gradients["value"].detach()
    acts = activations["value"].detach()
    weights = grads.mean(dim=(2, 3), keepdim=True)
    cam = (weights * acts).sum(dim=1).squeeze()
    cam = F.relu(cam).cpu().numpy()
    cam = cv2.resize(cam, (img.size[0], img.size[1]))
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-9)

    # Координаты прямоугольника
    bbox = get_cam_bbox(cam, threshold=0.5)

    # Снимаем хуки
    fwd_handle.remove()
    bwd_handle.remove()

    # Чистим память
    del input_tensor, outputs, grads, acts
    torch.cuda.empty_cache()

    return class_names[pred.item()], conf.item() * 100, bbox

def predict_folder_with_bbox(model, folder_path, device):
    results = []
    for fname in os.listdir(folder_path):
        if fname.lower().endswith((".png", ".jpg", ".jpeg")):
            path = os.path.join(folder_path, fname)
            label, confidence, bbox = predict_image_with_bbox(model, path, device)
            print(f"{fname}: {label} ({confidence:.2f}%), bbox={bbox}")
            results.append((fname, label, confidence, bbox))
    return results

In [5]:
model = models.densenet121(weights="IMAGENET1K_V1")
in_f = model.classifier.in_features
model.classifier = nn.Linear(in_f, len(class_names))
model.load_state_dict(torch.load("best_densenet_ct.pth", map_location=device))
model = model.to(device)

In [6]:
# Прогоняем папку
results = predict_folder_with_bbox(model, "postDICOM", device)

d:\Anaconda\envs\PythonAIPyTorch\Lib\site-packages\torch\nn\modules\module.py:1842: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


1.2.643.5.1.13.13.12.2.77.8252.00120800110906151309051502130401.png: COVID-19 (100.00%), bbox=(np.int64(126), np.int64(248), np.int64(370), np.int64(511))
1.2.643.5.1.13.13.12.2.77.8252.01001413041305061500030501001512.png: COVID-19 (100.00%), bbox=(np.int64(120), np.int64(339), np.int64(431), np.int64(511))
1.2.643.5.1.13.13.12.2.77.8252.01040806021000100410130007101215.png: COVID-19 (100.00%), bbox=(np.int64(154), np.int64(213), np.int64(414), np.int64(454))
1.2.643.5.1.13.13.12.2.77.8252.01050701051505050715080910060315.png: COVID-19 (100.00%), bbox=(np.int64(0), np.int64(0), np.int64(511), np.int64(511))
1.2.643.5.1.13.13.12.2.77.8252.01061114150915090104151114140611.png: COVID-19 (83.53%), bbox=(np.int64(0), np.int64(354), np.int64(289), np.int64(511))
1.2.643.5.1.13.13.12.2.77.8252.01070409071501051008110900111301.png: COVID-19 (99.74%), bbox=(np.int64(90), np.int64(177), np.int64(419), np.int64(511))
1.2.643.5.1.13.13.12.2.77.8252.01081511090912090508100510010313.png: COVID-19 (

In [7]:
import onnxruntime as ort

In [8]:
def export_to_onnx(model, onnx_file_path, input_shape=(1, 3, 224, 224)):
    """
    Экспортирует модель PyTorch в формат ONNX.

    Args:
        model (torch.nn.Module): Модель для экспорта.
        onnx_file_path (str): Путь, по которому будет сохранен ONNX-файл.
        input_shape (tuple): Ожидаемая форма входного тензора.
    """
    dummy_input = torch.randn(input_shape, requires_grad=True).to(device)
    torch.onnx.export(model,
                      dummy_input,
                      onnx_file_path,
                      export_params=True,
                      opset_version=11,
                      do_constant_folding=True,
                      input_names=['input'],
                      output_names=['output'],
                      dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
    print("Экспорт в ONNX завершен успешно!")
    print(f"Экспорт модели в ONNX формат: {onnx_file_path}")
    model.eval()


In [9]:
export_to_onnx(model, "densenet_ct.onnx")

Экспорт в ONNX завершен успешно!
Экспорт модели в ONNX формат: densenet_ct.onnx
